# Visual odometry
We can try to create a visual odometry part using feature matching and tracking between consecutive frames.
We use modified SIFT with RootSIFT to get the feature descriptors and KLT Tracker to get the odometry.

In [1]:
import cv2
import numpy as np
from reconstruction.dataset import Dataset
from reconstruction.image_matcher import ImageMatcher
from reconstruction.config import Config
import ipyplot
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

dataset = Dataset.read("data/3d-scans-cap-black-glossy-ha-2019-02-27T16_06_29")
matcher = ImageMatcher(None)

@widgets.interact(frames=widgets.IntRangeSlider(min=0, max=len(dataset.frames) - 1, value=[53, 56]))
def draw_matches(frames):
    source_img = cv2.imread(dataset.frames[frames[0]].color_file)
    target_img = cv2.imread(dataset.frames[frames[1]].color_file)

    source_keypoints, source_descriptors = matcher.detect_keypoints(source_img)
    target_keypoints, target_descriptors = matcher.detect_keypoints(target_img) 

    source_img_keypoints = cv2.drawKeypoints(source_img, source_keypoints, None)
    target_img_keypoints = cv2.drawKeypoints(target_img, target_keypoints, None)
    ipyplot.plot_images([source_img_keypoints, target_img_keypoints], img_width=300)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(source_descriptors, target_descriptors, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good.append(m)

    src_pts = np.float32([source_keypoints[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([target_keypoints[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    if len(dst_pts) < 4:
        display(Markdown("**Not enough matches to find homography**"))
        return 

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    matchesMask = mask.ravel().tolist()
    h, w = source_img.shape[:2]
    pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    target_img_keypoints = cv2.polylines(target_img_keypoints, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                    singlePointColor = None,
                    matchesMask = matchesMask, # draw only inliers
                    flags = 2)

    img_with_matches = cv2.drawMatches(source_img, source_keypoints, target_img, target_keypoints, good, None, **draw_params)
    plt.figure(figsize=(100,100))
    plt.imshow(img_with_matches)
    plt.axis(False)
    plt.show()

interactive(children=(IntRangeSlider(value=(53, 56), description='frames', max=194), Output()), _dom_classes=(…